In [1]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [31]:
df = pd.read_excel('D:/UNI/IA/RadioAnalyzer/audios/data_2.xlsx')

In [32]:
df.head()

,audio,categoria
0,BITEL_audio32_1,BITEL
1,BITEL_audio32_2,BITEL
2,BITEL_audio32_3,BITEL
3,BITEL_audio32_4,BITEL
4,BITEL_audio32_5,BITEL


In [33]:
def parser(row):
    file_name = 'D:/UNI/IA/RadioAnalyzer/audios/data/'+str(row.audio)+'.wav'
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=50).T,axis=0) 
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None
 
    feature = mfccs
    return feature

df['feature'] = df.apply(lambda row:parser(row),axis=1)

In [34]:
df.head()

,audio,categoria,feature
0,BITEL_audio32_1,BITEL,"[-189.7812790242206, 101.49177428657258, -23.8..."
1,BITEL_audio32_2,BITEL,"[-175.37935207679197, 73.70655624235172, -35.8..."
2,BITEL_audio32_3,BITEL,"[-182.69068488737872, 105.52600862618615, -22...."
3,BITEL_audio32_4,BITEL,"[-164.3735209101462, 87.70550743947643, -28.29..."
4,BITEL_audio32_5,BITEL,"[-134.1047263148321, 100.68159882374813, -24.3..."


In [35]:
df.feature.mean()

array([-141.61136486,   87.83948972,  -11.3172642 ,   36.91793859,
          6.02997049,   13.65196602,  -13.22049911,    9.32787953,
        -19.72403865,    7.01521117,  -11.40420601,    4.44072279,
         -9.29554603,    4.62122163,   -6.78208514,    4.54042141,
         -9.96980369,    5.03020277,   -8.29380947,    3.1613422 ,
         -7.63971253,    5.13898151,   -6.50189201,    4.91267608,
         -4.48113735,    2.99724316,   -3.7986776 ,    2.94617484,
         -3.10603016,    3.04356587,   -3.63088593,    3.29036947,
         -3.24486024,    2.62842151,   -2.14494122,    1.44768204,
         -1.18395566,    1.02736191,   -1.19728629,    0.5235764 ,
         -1.53232579,    1.21489912,   -0.5910588 ,    0.3081048 ,
         -0.25607287,   -0.34431625,    0.2881648 ,   -0.27160495,
          0.75587062,   -1.56005132])

In [36]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.utils import np_utils

X = np.array(df.feature.tolist())
Y = np.array(df.categoria.tolist())

lb = LabelEncoder()

y = np_utils.to_categorical(lb.fit_transform(Y))


num_labels = y.shape[1]
filter_size = 2

# build model
model = Sequential()

model.add(Dense(256, input_shape=(50,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [37]:
model.fit(X, y, batch_size=32, epochs=20)

Epoch 1/20
139/139 [==============================] - 0s 3ms/step - loss: 12.6295 - acc: 0.0719
Epoch 2/20
139/139 [==============================] - 0s 79us/step - loss: 10.8623 - acc: 0.2230
Epoch 3/20
139/139 [==============================] - 0s 72us/step - loss: 11.8378 - acc: 0.1511
Epoch 4/20
139/139 [==============================] - 0s 79us/step - loss: 11.4302 - acc: 0.1727
Epoch 5/20
139/139 [==============================] - 0s 87us/step - loss: 10.9441 - acc: 0.1799
Epoch 6/20
139/139 [==============================] - 0s 72us/step - loss: 11.3387 - acc: 0.2302
Epoch 7/20
139/139 [==============================] - 0s 79us/step - loss: 9.7096 - acc: 0.2662
Epoch 8/20
139/139 [==============================] - 0s 87us/step - loss: 9.4293 - acc: 0.3309
Epoch 9/20
139/139 [==============================] - 0s 79us/step - loss: 10.2835 - acc: 0.2302
Epoch 10/20
139/139 [==============================] - 0s 65us/step - loss: 9.3441 - acc: 0.3022
Epoch 11/20
139/139 [============

In [38]:
model.evaluate(X, y)

139/139 [==============================] - 0s 613us/step


[2.707785169503328, 0.7410071942446043]

In [39]:
predictes_Y = model.predict_classes(X)

In [40]:
predictes_Y_class = lb.inverse_transform(predictes_Y)

In [41]:
df['predicted'] = predictes_Y_class

In [42]:
df.head()

,audio,categoria,feature,predicted
0,BITEL_audio32_1,BITEL,"[-189.7812790242206, 101.49177428657258, -23.8...",MTC
1,BITEL_audio32_2,BITEL,"[-175.37935207679197, 73.70655624235172, -35.8...",MTC
2,BITEL_audio32_3,BITEL,"[-182.69068488737872, 105.52600862618615, -22....",MTC
3,BITEL_audio32_4,BITEL,"[-164.3735209101462, 87.70550743947643, -28.29...",MTC
4,BITEL_audio32_5,BITEL,"[-134.1047263148321, 100.68159882374813, -24.3...",MTC


In [26]:
df.to_csv("D:/UNI/IA/RadioAnalyzer/audios/out2.csv", index=False)

In [43]:
df['ok'] = (df['categoria']==df['predicted'])

In [44]:
df.head()

,audio,categoria,feature,predicted,ok
0,BITEL_audio32_1,BITEL,"[-189.7812790242206, 101.49177428657258, -23.8...",MTC,False
1,BITEL_audio32_2,BITEL,"[-175.37935207679197, 73.70655624235172, -35.8...",MTC,False
2,BITEL_audio32_3,BITEL,"[-182.69068488737872, 105.52600862618615, -22....",MTC,False
3,BITEL_audio32_4,BITEL,"[-164.3735209101462, 87.70550743947643, -28.29...",MTC,False
4,BITEL_audio32_5,BITEL,"[-134.1047263148321, 100.68159882374813, -24.3...",MTC,False


In [45]:
df['ok'].mean()

0.7410071942446043

In [46]:
dfoutput = df.drop(columns=['feature'])

In [47]:
dfoutput.to_csv("D:/UNI/IA/RadioAnalyzer/audios/out.csv", index=False)